<a href="https://colab.research.google.com/github/crossboww/Machine_Learning/blob/main/ML_learning/Model_Selection_in_ML_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing the dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
#Importing the Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

We are Working on Heart Disease Datasets

In [3]:
heart_data = pd.read_csv("heart.csv")

In [4]:
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
heart_data.shape

(303, 14)

In [6]:
heart_data.isnull().sum()

,0
age,0
sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0
oldpeak,0


Cheking the Distribution of Target Columns Values

In [7]:
heart_data['target'].value_counts()

,count
target,
1,165
0,138


1------- > Defective heart

0------- > Healthy heart

#Splitting the Features and Targets Columns

In [8]:
x = heart_data.drop(columns='target', axis = 1)
y = heart_data['target']

In [9]:
print(x)
print(y)

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  
0        0   0     1  
1        0   0     2  
2        2   0    

In [10]:
X = np.asarray(x)
Y = np.asarray(y)

#Model Selection

1.Comparing the model with defalut hyperparameter values using Cross-Validation.

In [11]:
#List of Models
models = [LogisticRegression(max_iter = 1000), SVC(kernel='linear'), KNeighborsClassifier(), RandomForestClassifier(random_state=0)]

In [12]:
def Compare_models_cross_validation():

  for model in models:
    cv_score = cross_val_score(model, X, Y, cv = 5)
    mean_accuracy = sum(cv_score) / len(cv_score)
    mean_accuracy = mean_accuracy * 100
    mean_accuracy = round(mean_accuracy, 2)

    print("Cross Validation Score for the", model, " = ", cv_score)
    print("Accuracy Score of the", model, "=", mean_accuracy, '%')
    print("-----------------------------------------------------")

In [13]:
Compare_models_cross_validation()

Cross Validation Score for the LogisticRegression(max_iter=1000)  =  [0.80327869 0.86885246 0.85245902 0.86666667 0.75      ]
Accuracy Score of the LogisticRegression(max_iter=1000) = 82.83 %
-----------------------------------------------------
Cross Validation Score for the SVC(kernel='linear')  =  [0.81967213 0.8852459  0.80327869 0.86666667 0.76666667]
Accuracy Score of the SVC(kernel='linear') = 82.83 %
-----------------------------------------------------
Cross Validation Score for the KNeighborsClassifier()  =  [0.60655738 0.6557377  0.57377049 0.73333333 0.65      ]
Accuracy Score of the KNeighborsClassifier() = 64.39 %
-----------------------------------------------------
Cross Validation Score for the RandomForestClassifier(random_state=0)  =  [0.85245902 0.90163934 0.81967213 0.81666667 0.8       ]
Accuracy Score of the RandomForestClassifier(random_state=0) = 83.81 %
-----------------------------------------------------


#2. Comparing the Model for different hyperparameters value using GridSearchCV

In [14]:
#Listing Models
models = [LogisticRegression(max_iter=10000), SVC(), KNeighborsClassifier(), RandomForestClassifier(random_state=0)]

In [15]:
#Creating the dictionary which contains hyperparameter of above models
param_grid = {
    "LogisticRegression" : {
        "model" : LogisticRegression(max_iter=10000),
        "params" : {
            "C" : [1,5,10,20]
        }
    },
    "SVC" : {
        "model" : SVC(),
        "params" : {
            "kernel" : ['linear', 'poly', 'rbf', 'sigmoid'],
            "C" : [1, 5, 10, 20]
        }
    },
    "KNeighborsClassifier" : {
        "model" : KNeighborsClassifier(),
        "params" : {
            "n_neighbors" : [3, 5, 10]
        }
    },
    "RandomForestClassifier" : {
        "model" : RandomForestClassifier(random_state=0),
        "params" : {
            "n_estimators" : [10, 20, 50, 100]
        }
    }
}



#Applying the GridSearchCv

In [19]:
result= []

for name, mp in param_grid.items():
  grid_search = GridSearchCV(mp['model'], mp['params'], cv=5)
  grid_search.fit(X, Y)
  grid_search.cv_results_

  result.append({
      "Model_name" : name,
      "Best_score" : grid_search.best_score_,
      "Best_params" : grid_search.best_params_
  })

results = pd.DataFrame(result, columns = ['Model_name', 'Best_score', 'Best_params'])
print(results)

               Model_name  Best_score                   Best_params
0      LogisticRegression    0.831585                      {'C': 5}
1                     SVC    0.828306  {'C': 1, 'kernel': 'linear'}
2    KNeighborsClassifier    0.643880            {'n_neighbors': 5}
3  RandomForestClassifier    0.838087         {'n_estimators': 100}
